# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from vacayconfig import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [7]:
# Read the WeatherPy_vacation.csv into a DataFrame
file_path = "../Vacation_Search/WeatherPy_vacation.csv"
vacation_df = pd.read_csv(file_path)

# Display sample data
vacation_df.tail()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
233,697,Pak Phanang,TH,82.42,overcast clouds,8.3511,100.2020,Green Park Inn Hotel
234,709,Kokopo,PG,85.87,overcast clouds,-4.3520,152.2633,Kokopo Beach Bungalow Resort
235,711,Ixtapa,MX,76.23,scattered clouds,20.7000,-105.2000,hotel la terminal
236,712,Luangwa,ZM,75.65,light rain,-15.6167,30.4167,Gerasimos Safari Lodge
237,713,Bac Lieu,VN,82.80,moderate rain,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu


## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [8]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [9]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [16]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start_end = vacation_df.loc[(vacation_df["City"] == "Kaeo")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Te Anau")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Tuatapere")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Mataura")]

## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [33]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start_end = vacation_start_end["Lat"].to_numpy()[0], vacation_start_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [21]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)
fig.add_layer(gmaps.traffic_layer())

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [22]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start_end,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,96,Kaeo,NZ,71.33,few clouds,-35.1000,173.7833,Bushido Lodge
1,178,Te Anau,NZ,71.20,broken clouds,-45.4167,167.7167,Kingsgate Hotel Te Anau
2,74,Tuatapere,NZ,70.70,broken clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
3,13,Mataura,NZ,73.06,few clouds,-46.1927,168.8643,Ellie's Villa


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [24]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat","Lng"]]
locations.head()

,Lat,Lng
0,-35.1000,173.7833
1,-45.4167,167.7167
2,-46.1333,167.6833
3,-46.1927,168.8643


In [31]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Define a new figure object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))